# Start Spark

In [1]:
import os
import sys
spark_home = os.environ['SPARK_HOME'] = '/Users/leiyang/Downloads/spark-1.6.0-bin-hadoop2.6/'
if not spark_home:
    raise ValueError('SPARK_HOME enviroment variable is not set')
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.9-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.9 (default, Dec 15 2014 10:37:34)
SparkContext available as sc, HiveContext available as sqlContext.


# k1,k2,v1,v2 : partition by k1 and sort by k2

In [7]:
%%writefile ss.txt
1,3,a,b
2,5,a,c
1,4,a,f
3,4,d,c
2,1,f,a
1,1,e,r
2,4,o,1
3,2,d,c
9,1,s,s

Overwriting ss.txt


In [8]:
def read_data(line):
    d = line.split(',')
    return [int(d[0]),int(d[1])],[d[2],d[3]]
data = sc.textFile("ss.txt").map(read_data)

In [9]:
data.collect()

[([1, 3], [u'a', u'b']),
 ([2, 5], [u'a', u'c']),
 ([1, 4], [u'a', u'f']),
 ([3, 4], [u'd', u'c']),
 ([2, 1], [u'f', u'a']),
 ([1, 1], [u'e', u'r']),
 ([2, 4], [u'o', u'1']),
 ([3, 2], [u'd', u'c']),
 ([9, 1], [u's', u's'])]

In [10]:
ssdata = data.repartitionAndSortWithinPartitions(numPartitions=3,partitionFunc= lambda x: x[0]%3,keyfunc=lambda x: x[1])

# Check ssdata by partition

In [11]:
ssdata.glom().collect()

[[([9, 1], [u's', u's']), ([3, 2], [u'd', u'c']), ([3, 4], [u'd', u'c'])],
 [([1, 1], [u'e', u'r']), ([1, 3], [u'a', u'b']), ([1, 4], [u'a', u'f'])],
 [([2, 1], [u'f', u'a']), ([2, 4], [u'o', u'1']), ([2, 5], [u'a', u'c'])]]